<a href="https://colab.research.google.com/github/NJiHyeon/Pytorch_for-deep-learning/blob/main/CH4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#기본모델 구축 + 저장

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader



---



### Load data set

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=8,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified




---



### Build a model

In [5]:
#모델을 만들 때 두가지를 import
import torch.nn as nn
import torch.nn.functional as F

#nn.Module을 상속 받는다.

class Net(nn.Module) : 
  def __init__(self) :
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x) :                    #x에 input이 들어온다.
    x = self.pool(F.relu(self.conv1(x)))   # 들어온 input이 convolution -> relu -> pool
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)                 # 사이즈를 변형할 때 view (이미지를 일자로 피는 작업)
    x = F.relu(self.fc1(x))                # linear연산 -> relu
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net = Net()



###### - __init__부분에서 내가 사용하고자 하는 연산들을 정의한다.(위의 두 줄은 그대로 밑에 부분부터는 원하는 것을 작성)
###### - forward 함수는 연산의 순서를 정해주는 부분
###### - 모델이 class이기 때문에 class에 대한 인스턴스(net=Net())만든다.이제 net을 가지고 모델을 돌리게 된다.

In [6]:
print(net)
#32 -> 28 -> 14 -> 10 -> 5

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)




---



### Implement the model with training set

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

###### - supervised running에서는 output이 나오고 -> output을 가지고 loss를 구하고 -> loss를 바탕으로해서 gradient값을 구하고 -> 다시 그 모델에 있는 파라미터들을 업데이트 시켜주는 형태
###### - loss function과 gradient를 구할 수 있는 optimizer를 정의
###### - 여기서 사용한 loss(nn.CorssEntropyLoss())는 전형적인 분류 문제에 쓰이는 함수
###### - net.parameters() : net안에 있는 파라미터들을 모아놓은 것(파라미터를 업데이터 시켜준다라고 생각하기)
###### - scheduling을 통해 lr을 가변적으로 운영 가능하다.
###### 정리하자면, optimizer와 loss function을 정의해주기

In [8]:
# 실질적인 학습 부분
for epoch in range(1) :                 # loop over the dataset multiple times
                                        # 데이터 전체를 몇번 봤는가

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0) :
    # get inputs : data is a list of [inputs, labels](이미지, 레이블 데이터)
    inputs, labels = data

    #옵티마이저 초기화 
    optimizer.zero_grad()

    # forwoard + backward + optimizer
    outputs = net(inputs)               #이미지(inputs)를 모델(net)에 넣어서 산출된 값 output
    loss = criterion(outputs, labels)   # output과 실제 labels과 비교해서 loss 계산
                                        # crossentropy에 의해 loss값 계산 
    loss.backward()                     # loss 기준으로 gradient 계산하게 되므로 loss를 기준으로 backward한다고 정의
    optimizer.step()                    # optimizer 계산

    # print statistics
    running_loss += loss.item()         # 숫자 하나의 형태로 바꾸어 숫자를 계속 더해 누적시키는 형태
    if i % 2000 == 1999 :               # print every 2000 mini_batches
      print('[%d, %5d] loss : %.3f' %
            (epoch + 1, i+1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

[1,  2000] loss : 2.180
[1,  4000] loss : 1.783
[1,  6000] loss : 1.595
Finished Training


###### - loss 값이 작아지는 것으로 보아 학습은 잘 되고 있다.




---



### Save the trained model
 - 저장을 하지 않으면 이 값을 원할 때 학습을 한번 더 해 주어야 하는데 데이터가 많아지고 epoch이 많아지면 시간이 많이 소요
 - 따라서 save를 잘 해야 한다.

In [9]:
PATH = './clfar_net.pth'
torch.save(net.state_dict(), PATH)



---



### Load the pre-trained model

In [10]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [12]:
# test data를 가지고 예측하는 방법
correct = 0
total = 0
with torch.no_grad() :                            # test할 때에는 gradient update를 하지 않는다는 의미
  for data in testloader :                        # testloader도 배치로 나누었기 때문에 for문 이용
    images, labels = data
    outputs = net(images)                         # 우리의 모델 net에 images 넣기, loss는 계산 X, output에는 10개의 값들이 있는 벡터 출력
    _, predicted = torch.max(outputs.data, 1)     # 그 중에서 가장 큰 것으로 예측 하겠다는 의미(값이 10개에서 1개로 바뀜)
    total += labels.size(0)
    correct += (predicted == labels).sum().item() # label도 하나이기 때문에 predicted랑 label이랑 같으면 맞은것으로 한다.

print('Accuracy of the network on the 10000 test images : %d %%' % (
    100* correct / total))



Accuracy of the network on the 10000 test images : 45 %


###### - 지난 시간에는 데이터를 불러오는 방법, 전처리 하는 방법
###### - 이번 시간에는 모델 만드는데 심플한 모델 만들기(실질적인 학습에 해당되는 for문 작성), save, load하는 방법
###### - 다음 시간에는 epoch이 높다고 해서 무조건 학습이 잘 되었다고 할 수는 없다. 학습하다가 중간에서 best model이 나올 수 있기 때문에 for문 안에 조건문을 넣어서 저장하도록 할 수 있다. 
###### - 일반적인 모델의 텐서는 CPU 연산을 할 수 있는 텐서인데 이미지 처리는 CPU로 하게 되면 매우 오래 걸리므로 GPU 연산으로 해야한다.
###### - 트랜스퍼 러닝에 대해서